# Raster Exposure — utility runner
_Generated on 2025-09-14 15:44 UTC_

## Setup & Config

In [ ]:
# --- Environment & Imports ---
import os, sys, glob, json
from pathlib import Path

# Make sure both the repo root and a potential 'modules/' folder are importable
ROOT = Path.cwd()
if (ROOT / "modules").exists():
    sys.path.insert(0, str(ROOT / "modules"))
sys.path.insert(0, str(ROOT))

# Basic utilities
import matplotlib.pyplot as plt

# YAML config loader (fallback if modules.config_utils not found)
try:
    from modules.config_utils import load_config  # project helper if available
except Exception:
    import yaml
    def load_config(path):
        with open(path, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)

# Helper to auto-detect a config file if not set
DEFAULT_CONFIG_CANDIDATES = ["config.yaml", "config.yml", "configs/config.yaml"]
for _cand in DEFAULT_CONFIG_CANDIDATES:
    if Path(_cand).exists():
        CONFIG_PATH = _cand
        break
else:
    # Put a placeholder if nothing found
    CONFIG_PATH = "config.yaml"  # adjust if needed
print(f"Using CONFIG_PATH = {CONFIG_PATH!r}")
CONFIG = load_config(CONFIG_PATH)
CONFIG

## Run generic raster exposure

In [ ]:
try:
    from modules.raster_exposure import process_raster_exposures
except Exception:
    from raster_exposure import process_raster_exposures

import geopandas as gpd

aoi_path = CONFIG.get("aoi")
aoi_gdf = gpd.read_file(aoi_path) if aoi_path else None

# If your CONFIG keeps per-type dicts somewhere, you can pass them directly.
# Otherwise, you can load single layers and the function may handle dict creation internally, depending on your implementation.
points_by_type = CONFIG.get("_points_by_type")  # set by your pipeline if cached
lines_by_type  = CONFIG.get("_lines_by_type")

if points_by_type is None or lines_by_type is None:
    # Optional: fall back to reading generic inputs
    pts_path = CONFIG.get("inputs", {}).get("points")
    lns_path = CONFIG.get("inputs", {}).get("lines")
    points_by_type = {"infra": gpd.read_file(pts_path)} if pts_path else {}
    lines_by_type  = {"infra": gpd.read_file(lns_path)} if lns_path else {}

res = process_raster_exposures(
    config=CONFIG,
    aoi=aoi_gdf,
    points_by_type=points_by_type,
    lines_by_type=lines_by_type,
    sample_points_per_line=int(CONFIG.get("sample_points_per_line", 10))
)
print("Exposure results keys:", list(res.keys()))

In [ ]:
# --- Browse outputs ---
out_dir = Path(CONFIG.get("output_dir", "output"))
out_dir.mkdir(parents=True, exist_ok=True)
maps = sorted(list(out_dir.rglob("*.png"))) + sorted(list(out_dir.rglob("*.jpg")))
rasters = sorted(list(out_dir.rglob("*.tif"))) + sorted(list(out_dir.rglob("*.tiff")))
vectors = sorted(list(out_dir.rglob("*.gpkg"))) + sorted(list(out_dir.rglob("*.geojson")))

print(f"Found {len(maps)} map images, {len(rasters)} rasters, {len(vectors)} vectors under {out_dir}")

# Show the latest map if available
if maps:
    from PIL import Image
    display(Image.open(maps[-1]))
else:
    print("No map images found yet.")